<a href="https://colab.research.google.com/github/nimrashaheen001/Programming_for_AI/blob/main/BasepaperImplementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install nibabel

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
import os
import nibabel as nib  # For potential metadata extraction (if needed)
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from PIL import Image  # For image loading
from torchvision import transforms # For image transformations
#Mount Google Drive (uncomment this in Google Colab)
from google.colab import drive
drive.mount('/content/drive')

class BasicBlock2D(nn.Module):  # Changed class name to BasicBlock2D
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock2D, self).__init__()
        # Changed to 2D convolutional layers
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        # Changed to 2D batch normalization
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion * planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class ResNet2D(nn.Module):  # Changed class name to ResNet2D
    def __init__(self, block, num_blocks, in_channels=3, num_classes=4):  # Updated in_channels and num_classes
        super(ResNet2D, self).__init__()
        self.in_planes = 64

        # Initial convolution layer (changed to 2D)
        self.conv1 = nn.Conv2d(in_channels, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)  # Changed to 2D batch normalization

        # ResNet layers (using BasicBlock2D)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)

        # Classification head (changed to 2D)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))  # Changed to 2D adaptive average pooling
        self.fc = nn.Linear(512 * block.expansion, 512)

        # Multiple output heads (adjusted for 4 classes)
        self.classification_head = nn.Linear(512, num_classes)
        self.regression_mmse = nn.Linear(512, 1)  # MMSE score regression
        self.regression_cdr = nn.Linear(512, 1)   # Clinical Dementia Rating regression

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        # Input processing
        out = F.relu(self.bn1(self.conv1(x)))

        # ResNet blocks
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)

        # Global pooling
        out = self.avgpool(out)
        out = out.view(out.size(0), -1)

        # Shared features
        features = F.relu(self.fc(out))

        # Multi-task outputs
        classification = self.classification_head(features)
        mmse_score = self.regression_mmse(features)
        cdr_score = self.regression_cdr(features)

        return {
            'classification': classification,
            'mmse_score': mmse_score,
            'cdr_score': cdr_score,
            'features': features
        }

def ResNet18_2D(in_channels=3, num_classes=4):  # Changed function name and defaults
    return ResNet2D(BasicBlock2D, [2, 2, 2, 2], in_channels, num_classes)  # Us

class BrainMRIDataset(Dataset):
    def __init__(self, data_dir, classes=['VeryMildDemented', 'MildDemented', 'NonDemented', 'ModerateDemented'], transform=None):
        """
        Dataset for multimodal brain MRI data

        Args:
            data_dir (str): Directory containing the data
            classes (list): List of class names
            transform (callable, optional): Optional transform to be applied on a sample
        """
        self.data_dir = data_dir
        self.classes = classes
        self.transform = transform

        # Find all images and labels
        self.images = []
        self.labels = []

        # Assuming directory structure: data_dir/class_label/image.png (or other image format)
        for class_idx, class_name in enumerate(classes):
            class_path = os.path.join(data_dir, class_name)
            image_files = [f for f in os.listdir(class_path) if os.path.isfile(os.path.join(class_path, f))]

            for image_file in image_files:
                image_path = os.path.join(class_path, image_file)
                self.images.append(image_path)
                self.labels.append(class_idx) # Assign class index as label

        print(f"Found {len(self.images)} images in total.")

    def __getitem__(self, idx):
        image_path = self.images[idx]
        label = self.labels[idx]

        # Load image
        image = Image.open(image_path).convert('RGB')  # Convert to RGB if needed

        # Apply transformations
        if self.transform:
            image = self.transform(image)

        # Simulate or load mmse_score and cdr_score (Replace with your actual logic)
        # Here I'm simulating them based on the label for demonstration purposes
        mmse_score = torch.tensor([28.0 if label == 2 else 20.0 + np.random.normal(0, 2)], dtype=torch.float32) # Assuming label 2 is 'NonDemented'
        cdr_score = torch.tensor([0.0 if label == 2 else 1.0 + np.random.normal(0, 0.5)], dtype=torch.float32)   # Assuming label 2 is 'NonDemented'


        return {'image': image, 'label': torch.tensor(label, dtype=torch.long), 'mmse_score': mmse_score, 'cdr_score': cdr_score}

    def __len__(self):
        """
        Returns the number of samples in the dataset.
        """
        print("Calling __len__ function!")  # Debug print statement
        return len(self.images)

# ... (Rest of the code) ...

def train_model(model, dataloaders, criterion_dict, optimizer, scheduler, num_epochs=25, device='cuda'):
    model = model.to(device)

    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0
            running_mmse_loss = 0.0
            running_cdr_loss = 0.0

            # Iterate over data
            for inputs in tqdm(dataloaders[phase]):
                images = inputs['image'].to(device)
                labels = inputs['label'].to(device)
                mmse_scores = inputs['mmse_score'].to(device)
                cdr_scores = inputs['cdr_score'].to(device)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward pass
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(images)
                    _, preds = torch.max(outputs['classification'], 1)

                    # Compute losses
                    classification_loss = criterion_dict['classification'](outputs['classification'], labels)
                    mmse_loss = criterion_dict['regression'](outputs['mmse_score'], mmse_scores)
                    cdr_loss = criterion_dict['regression'](outputs['cdr_score'], cdr_scores)

                    # Combined loss
                    loss = classification_loss + 0.5 * mmse_loss + 0.5 * cdr_loss

                    # Backward + optimize only in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Statistics
                running_loss += loss.item() * images.size(0)
                running_corrects += torch.sum(preds == labels.data)
                running_mmse_loss += mmse_loss.item() * images.size(0)
                running_cdr_loss += cdr_loss.item() * images.size(0)

            if phase == 'train' and scheduler is not None:
                scheduler.step()

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)
            epoch_mmse_loss = running_mmse_loss / len(dataloaders[phase].dataset)
            epoch_cdr_loss = running_cdr_loss / len(dataloaders[phase].dataset)

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f} MMSE Loss: {epoch_mmse_loss:.4f} CDR Loss: {epoch_cdr_loss:.4f}')

            # Deep copy the model if it's the best validation accuracy so far
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict().copy()

    print(f'Best val Acc: {best_acc:4f}')

    # Load best model weights
    model.load_state_dict(best_model_wts)
    return model

def main():
    # Set device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

 # Set paths
    data_dir = "/content/drive/MyDrive/Alzheimer_MRI_4_classes_dataset"  # Update with your Google Drive path

    # Define transformations (resize, normalize, etc.)
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resize to a common size
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize using ImageNet stats
    ])

    # Create dataset
    # Create dataset
    dataset = BrainMRIDataset(data_dir=data_dir, classes=['VeryMildDemented', 'MildDemented', 'NonDemented', 'ModerateDemented'], transform=transform)
    # Split into train and validation sets
    train_indices, val_indices = train_test_split(
        range(len(dataset)),
        test_size=0.2,
        random_state=42,
        stratify=dataset.labels
    )

    train_dataset = torch.utils.data.Subset(dataset, train_indices)
    val_dataset = torch.utils.data.Subset(dataset, val_indices)

    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=4)
    val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False, num_workers=4)

    dataloaders = {
        'train': train_loader,
        'val': val_loader
    }

   # Create model
    model = ResNet18_2D(in_channels=3, num_classes=4)  # Update in_channels and num_classes for 2D images and 4 classes
    # ... (Loss, optimizer, training, saving - Similar a

    # Define loss functions
    criterion_dict = {
        'classification': nn.CrossEntropyLoss(),
        'regression': nn.MSELoss()
    }

    # Optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    # Learning rate scheduler
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

    # Train model
    model = train_model(
        model=model,
        dataloaders=dataloaders,
        criterion_dict=criterion_dict,
        optimizer=optimizer,
        scheduler=scheduler,
        num_epochs=25,
        device=device
    )

    # Save model
    torch.save(model.state_dict(), 'brain_mri_model.pth')

    print("Training complete!")

def predict(model, dataloader, device='cuda'):
    model.eval()
    model = model.to(device)

    results = {
        'subject_ids': [],
        'true_labels': [],
        'predictions': [],
        'mmse_scores': [],
        'cdr_scores': []
    }

    with torch.no_grad():
        for inputs in tqdm(dataloader):
            images = inputs['image'].to(device)
            labels = inputs['label'].cpu().numpy()
            subject_ids = inputs['subject_id']

            outputs = model(images)
            _, preds = torch.max(outputs['classification'], 1)

            # Store results
            results['subject_ids'].extend(subject_ids)
            results['true_labels'].extend(labels)
            results['predictions'].extend(preds.cpu().numpy())
            results['mmse_scores'].extend(outputs['mmse_score'].cpu().numpy().flatten())
            results['cdr_scores'].extend(outputs['cdr_score'].cpu().numpy().flatten())

    return results

if __name__ == "__main__":
    main()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Using device: cuda


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Found 6400 images in total.
Calling __len__ function!
Epoch 0/24
----------


100%|██████████| 1280/1280 [04:45<00:00,  4.49it/s]


train Loss: 10.8677 Acc: 0.4854 MMSE Loss: 19.0536 CDR Loss: 0.4276


...

val Loss: 6.3210 Acc: 0.6281 MMSE Loss: 10.7665 CDR Loss: 0.2865
Epoch 24/24
----------


100%|██████████| 1280/1280 [03:28<00:00,  6.12it/s]


train Loss: 6.3565 Acc: 0.6256 MMSE Loss: 10.7592 CDR Loss: 0.3074


100%|██████████| 320/320 [00:17<00:00, 18.35it/s]

val Loss: 6.6636 Acc: 0.6203 MMSE Loss: 11.4279 CDR Loss: 0.3124
Best val Acc: 0.628125
Training complete!
